In [ ]:
import re
import numpy as np

In [ ]:
class EventDetection:

  def __init__(self):

    def read_file(file):
        texts = []
        for word in file:
            text = word.rstrip('\n')
            texts.append(text)
        return texts

    agreement_ = read_file(open('sources/agreement.txt', 'r', encoding = 'utf-8').readlines())
    contract_ = read_file(open('sources/contract.txt', 'r', encoding = 'utf-8').readlines())
    removal_ = read_file(open('sources/removal_installation.txt', 'r', encoding = 'utf-8').readlines())
    postfix = read_file(open('sources/postfix.txt', 'r', encoding = 'utf-8').readlines())
    country = read_file(open('sources/country.txt', 'r', encoding = 'utf-8').readlines())
    sport_keyword = read_file(open('sources/sport_key_words.txt', 'r', encoding = 'utf-8').readlines())
    geog_keywords = read_file(open('sources/countries_cities.csv', 'r', encoding = 'utf-8').readlines())
    iran_geog_keywords = read_file(open('sources/city_iran.csv', 'r', encoding = 'utf-8').readlines())
    #max_allowded_space = 5

    self.postfixs = r'\b(' + ''.join([i+'|' for i in postfix])[:-1] + r')\b'
    self.contract_verbs = r'\b(' + ''.join([i+'|' for i in contract_])[:-1]  + r')\b'
    self.removal_verbs = r'\b(' + ''.join([i+'|' for i in removal_])[:-1]  + r')\b'
    self.agreement_verbs = r'\b(' + ''.join([i+'|' for i in agreement_])[:-1]  + r')\b'
    self.countries = r'\b(' + ''.join([i+'|' for i in country])[:-1]  + r')\b'
    #print(self.agreement_verbs)

  def run (self, input):
    
    res = self.agreement(input, self.postfixs, self.agreement_verbs, self.countries)
    for output in res : 
      if len(output["text"]) > 5:
        print(output)

    res = self.Contract(input, self.postfixs, self.contract_verbs, self.countries)
    for output in res : 
      if len(output["text"]) > 5:
        print(output)

    res = self.removal_installation(input, self.postfixs, self.removal_verbs, self.countries)
    for output in res : 
      if len(output["text"]) > 5:
        print(output)

    res = self.price_change(input)
    for output in res : 
      if len(output["text"]) > 5:
        print(output)

    res = self.import_export(input)
    for output in res : 
      if len(output["text"]) > 5:
        print(output)

    res = self.death(input)
    for output in res : 
      if len(output["text"]) > 5:
        print(output)

  def agreement(self, input, postfix, key_verbs, key_countries):

    key_words = r"\b(مذاکر|توافق|گفتگو|گفت و گو|گفت‌و‌گو|)\s*(ه|ات|ها|های)*\b"
    parties  = f"(بین|میان)?\s*{key_countries}\s*و\s*{key_countries}\s*(و\s*{key_countries}\s*)*"
    description = r"\b(بر سر|درمورد)\b"
    names = r"\b(صلح|آتش بس|تحریم|تحریم های|اولیه|نهایی|همکاری|چندجانبه|چند جانبه|دو طرفه|مربوط به|صلح جهانی|برجام|مفاد|عهد نامه|اقتصادی|نظامی|جنگی)\b"
    pattern_1= f"{parties}\s*({description}\s*)?({names}\s*)*{key_verbs}\s*({postfix}\s*)+"
    pattern_2 = f"{key_words}\s*({names}\s*)*({key_countries}\s*)*{parties}?({description}\s*)?({names}\s*)*"
    pattern = f"({pattern_1})|({pattern_2})"

    output = []
    for m in re.finditer(pattern, input):
        start, end = m.span()
        output.append({'type':'گفتگو و مذاکرات و توافق', 'text':input[start:end], 'span':[start, end], 'place':'', 'time':''})
    return output

  
  def Contract(self, input, postfix, key_verbs, key_countries):

    starter = r"\b(امضاء|امضای|عقد|اجرای|انعقاد|لغو|توافق بر سر)\b"
    key_words = r"\b(پیمان|قرارداد|عهد نامه|قطع نامه|تعهد نامه)\b"
    parties  = f"(بین|میان)?\s*{key_countries}\s*و\s*{key_countries}\s*(و\s*{key_countries}\s*)*"
    description = r"\b(را|بر سر|درمورد)\b"
    names = r"\b(صلح|آتش بس|تحریم|تحریم های|اولیه|نهایی|همکاری|چندجانبه|چند جانبه|دو طرفه|مربوط به|صلح جهانی|برجام|مفاد|عهد نامه|نظامی|جنگی|اقتصادی|برون مرزی|بین المللی|منطقه ای|سازمان ملل)\b"
    pattern_1= f"{parties}\s*({description}\s*)?({key_words}\s*)*([0-9]+\s*)*({names}\s*)*({description}\s*)?({key_verbs})\s({postfix}\s*)+"
    pattern_2 = f"{starter}\s*{key_words}\s*([0-9]+\s*)*({names}\s*)*{parties}?"
    pattern = f"({pattern_1})|({pattern_2})"

    output = []
    for m in re.finditer(pattern, input):
        start, end = m.span()
        output.append({'type':'قرار داد های رسمی', 'text':input[start:end], 'span':[start, end], 'place':'', 'time':''})
    return output


  def removal_installation(self, input, postfix, key_verbs, key_countries):
  
    key_words = r"\b(عزل|نصب|استعفا|استعفای|انتخاب|استیضاح|برکناری|جایگزینی|انتصاب|کناره گیری)\b"
    key_positions = r"\b(نسخت وزیر|جمهور|ریاست|جمهوری|نخست وزیر|نخست‌وزیر|رئیس|سپاه|ارتش|ارگان|اداره|ادارات|رهبر|مقام|منصب|فرمانده|مسئولیت|کشور|جمهور|مسئول|مجلس|قوه|قضائیه|مجریه|مقننه|ملکه|پادشاه|ولیعهد|جانشین|سمت|مقام|سازمان|ملل)\b"
    description = r"\b(را|برای|به عنوان|از|خود|خویش|به)\b"

    pattern_1= f"{key_words}\s*({key_positions}\s*)+({key_countries}\s*)*({description}\s*)*({key_countries}\s*)*({description}\s*)*"
    pattern_2 = f"({key_positions}\s*)+({key_countries}\s*)*({description}\s*)*({key_positions}\s*)*({key_countries}\s*)*({description}\s*)*{key_verbs}\s*({postfix}\s*)+"
    pattern = f"({pattern_1})|({pattern_2})"

    output = []
    for m in re.finditer(pattern, input):
        start, end = m.span()
        output.append({'type':'عزل و نصب و استعفا و انتخاب', 'text':input[start:end], 'span':[start, end], 'place':'', 'time':''})
    return output


  def price_change(self, input):
    change_infinitive = r"\b(افزایش|کاهش|رشد|صعود|نزول|بالا رفتن|پایین آمدن|[\w\u200c]+ برابر شدن)\b"
    percentage_p1 = r"\b([\w\u200c]+ درصدی)\b"
    value_keyword = r"\b(قیمت|ارزش|بها|بهای)\b"
    products = r"([\w\u200c]+((، [\w\u200c]+)*( و [\w\u200c]+))?)"
    change_keyword = r"\b(افزایش|کاهش|صعود|گران|ارزان|نزول|بالا|پایین|[\w\u200c]+ برابر)\b"
    change_verb = r"\b(یافت|می یابد|یافته است|کرد|می کند|کرده است|رفت|می رود|رفته است|آمد|می آید|آمده است|شد|می شود|شده است)\b"
    percentage_p2 = r"\b([\w\u200c]+ درصد)\b"

    pattern_1 = f"(({change_infinitive}\s*({percentage_p1}\s*)?{value_keyword}\s*)|((گران شدن|ارزان شدن)\s*({percentage_p1}\s*)?)){products}"
    pattern_2 = f"({value_keyword}\s*)?{products}\s*({percentage_p2}\s*)?{change_keyword}\s*{change_verb}"
    pattern = f"({pattern_1})|({pattern_2})"

    output = []
    for m in re.finditer(pattern, input):
        start, end = m.span()
        output.append({'type':'تغییر قیمت', 'text':input[start:end], 'span':[start, end], 'place':'', 'time':''})
    return output


  def import_export(self, input):
    change_keywords = r"\b(افزایش|کاهش|صعود|نزول|بالا رفتن|پایین آمدن|[\w\u200c]+ برابر)\b"
    inex_infinitive = r"\b(واردات|صادرات|وارد کردن|صادر کردن)\b"
    adjectives = r"\b(بی رویه|ناکافی|بیش از اندازه)\b"
    products = r"([\w\u200c]+((، [\w\u200c]+)*( و [\w\u200c]+))?)"
    sord = r"\b((از|به)\s*[\w\u200c]+)\s*\b"
    inex_keywords = r"\b(صادر|وارد)\b"
    inex_verbs = r"\b(شد|شدند|می‌شود|می‌شوند|شده است|شده‌اند|کرد|کردند|می‌کند|می‌کنند|کرده است|کرده‌اند)\b"

    pattern_1 = f"({change_keywords}\s*)?{inex_infinitive}\s*({adjectives}\s*)?{products}(\s*{sord})*"
    pattern_2 = f"{products}\s*(را)?\s*({sord})*\s*{inex_keywords}\s*{inex_verbs}"
    pattern = f"({pattern_1})|({pattern_2})"

    output = []
    for m in re.finditer(pattern, input):
        start, end = m.span()
        output.append({'type':'واردات و صادرات', 'text':input[start:end], 'span':[start, end], 'place':'', 'time':''})
    return output


  def death(self, input):
    death_infinitive = r"\b(وفات|مرگ|فوت|درگذشت|شهادت|عروج|جان باختن|کشته شدن|به قتل رسیدن|از دست رفتن)\b"
    death_verb = r"\b(فوت کرد|مرد|درگذشت|به شهادت رسید|شهید شد|به دیار باقی شتافت|جان باخت|جان به جان آفرین تسلیم کرد|کشته شد|به قتل رسید|دار فانی را وداع گفت|به دیدار حق شتافت|را از دست دادیم)(ند)?\b"
    people = r"([\w\u200c]+ تن|[\w\u200c]+ نفر)\s*(از [\w\u200c]+)?"
    description = r"\b(امام|آقای|خانم|آیت‌الله)\b"
    prayers = r"\b(رحم الله علیه|علیه السلام)\b"
    names = r"[\w\u200c]+"

    pattern_1 = f"{death_infinitive}\s*(({description}\s*)?{names}(\s*{prayers})?|{people})"
    pattern_2 = f"(({description}\s*)?{names}\s*({prayers})|{people})\s*{death_verb}"
    pattern = f"({pattern_1})|({pattern_2})"

    output = []
    for m in re.finditer(pattern, input):
        start, end = m.span()
        output.append({'type':'مرگ', 'text':input[start:end], 'span':[start, end], 'place':'', 'time':''})
    return output




In [ ]:
detector = EventDetection()
#test = 'باافزایش واردات گوشت، گوشت ارزان شد و مردم به خیابان ها سرازیر شدند و در همهمه دو تن از هموطنان درگذشتند. پس از این سانحه، گوشت از ایران به برزیل صادر شد.'
#detector.run(test)
#test = 'با افزایش 50 درصدی قیمت گوشت، نان و ماست، مردم به خیابان ها سرازیر شدند. پس از این سانحه قیمت پانسمان افزایش یافت.'
#detector.run(test)
#test = 'گفتگوهای صلح اوکراین میان اوکراین و روسیه از سر گرفته‌ خواهد شد.'
#detector.run(test)
#test = 'امروز مورخ 24 فروردین 1401 مذاکرات برجام میان ایران و پنج به علاوه یک به پایان رسید و ایران و روسیه بر سر مفاد برجام به توافق رسیدند.'
test = "مذاکرات چندجانبه میان اتحادیه اروپا و ایران و آمریکا درمورد برجام بی نتیجه ماند."
detector.run(test)

{'type': 'گفتگو و مذاکرات و توافق', 'text': 'مذاکرات چندجانبه میان اتحادیه اروپا و ایران ', 'span': [0, 44], 'place': '', 'time': ''}
